In [41]:
import pickle
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
from transformers import AutoTokenizer, RobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader

from moviepy.editor import *

from typing import Tuple, List, Dict

In [42]:
VIDEO_PATH = Path('../data') / 'urfunny2_video'
AUDIO_PATH = Path('../data') / 'urfunny2_audio'
DATA_PATH = Path('../sdk_features/')

In [46]:
target_dict = load_pickle(DATA_PATH / "humor_label_sdk.pkl")
for idx in target_dict.keys():     
    video = VideoFileClip(f'../data/urfunny2_video/{idx}.mp4')
    video.audio.write_audiofile(f'../data/urfunny2_audio/{idx}.mp3')

MoviePy - Writing audio in ../data/urfunny2_audio/1.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/3.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/4.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/5.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/7.mp3


...

MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/15615.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/15616.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/15618.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/urfunny2_audio/15619.mp3


MoviePy - Done.


In [43]:
def load_pickle(pickle_file) -> Dict:
    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError as e:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print('Unable to load data ', pickle_file, ':', e)
        raise
    return pickle_data

In [53]:
class HumorDataset(Dataset):

    def __init__(self):

        self.origin_text = load_pickle(DATA_PATH / "language_sdk.pkl")
        self.preprocessed_text = []
        for idx in self.origin_text:
            language_feats = self.origin_text[idx]['context_sentences'] + \
                [self.origin_text[idx]['punchline_sentence']]
            language_feats = list(
                map(lambda i: ' ' + language_feats[i], range(0, len(language_feats))))
            language_feats = '.'.join(language_feats)
            self.preprocessed_text.append(language_feats)

        target_dict = load_pickle(DATA_PATH / "humor_label_sdk.pkl")
        self.videos = [VIDEO_PATH / f'{idx}.mp4' for idx in target_dict.keys()]
        self.audio = [AUDIO_PATH / f'{idx}.mp3' for idx in target_dict.keys()]
        self.target = list(target_dict.values())

    def __len__(self):
        return len(self.language_feats)

    def __getitem__(self, index: int) -> Tuple[str, int]:
        return self.preprocessed_text[index], self.videos[index], self.audio[index], self.target[index]

In [54]:
dataset = HumorDataset()

In [55]:
dataset[0]

(" the mathematics of quantum mechanics very accurately describes how our universe works. and it tells us our reality is continually branching into different possibilities just like a coral. it's a weird thing for us humans to wrap our minds around since we only ever get to experience one possibility. this quantum weirdness was first described by erwin and his cat. the cat likes this version better",
 WindowsPath('../data/urfunny2_video/1.mp4'),
 WindowsPath('../data/urfunny2_audio/1.mp3'),
 1)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

num_labels = len(model.config.id2label)
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion", num_labels=num_labels)

labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)